<a href="https://colab.research.google.com/github/dnhshl/cc-ai/blob/main/iris-neural-network.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Iris Neural Network

## Introduction

Artifical Neural Networks are quickly becoming one of the most popular and widely used mechanisms in Machine Learning and Data Analysis. In the last number of years, numerous libraries and software has been developed to equip programmers with a set of tools for modeling and analysing data in order to recognise patterns and make predictions using large data sets. In today's age of [Big Data](https://en.wikipedia.org/wiki/Big_data) it is important to try make sense of all of the data we have in society. This could range from social media pattern recognitions from anything to finance and economic trends. The reality is that today we have more data in existence than ever before and it growing at a vast and exponential rate.

Artifical Neural Networks aim to mimic and replicate the neurons of a human brain and using the power of the complex mathematical functions allow us to process and model data in such a way that we can form rational assumptions on a given data set.

Given the sheer amount of data out there it is important to note that data we may analyse is often subject to human error and may not always hold a valid essense of truth. For the purpose of this example we will take a look at the [Iris Data Set](https://archive.ics.uci.edu/ml/datasets/iris).

More information on the data set can be found on the link provided above or on the front page [README of this repository](https://github.com/damiannolan/iris-neural-network).

Throughout the notebook we aim to build an Artifical Neural Network capable of making predictions of species of Iris Flowers using [Keras](https://keras.io) - Keras is a high-level neural networks API, written in Python and capable of running on top of [Tensorflow](https://www.tensorflow.org/).

So without further ado, lets get started!

## Importing the data set

In [ ]:
from sklearn import datasets
import pandas as pd

# Load the Iris dataset
iris = datasets.load_iris()

# Convert the dataset to a pandas DataFrame for easier manipulation
iris_df = pd.DataFrame(data=iris.data, columns=iris.feature_names)
iris_df['target'] = iris.target

# Display the first few rows of the DataFrame
print(iris_df.head())


## Inputs and Outputs
### Data Investigation and Classification

Before trying to create a model for our Neural Network we first need to investigate our data and determine what will be the inputs and what will be our outputs. The CSV file provided contains 5 columns with:

- Sepal Length
- Sepal Width
- Petal Length
- Petal Width
- Species

Judging by the fact that we are trying to make predictions we must split our data set into sets of:

- Inputs - Numerical data values
- Outputs - Classification of Iris Flower species


Now that we have the data set loaded we can extract the data we need into appropriate data sets in preparation for training and testing our Model.

In [ ]:
import numpy as np
import keras as kr

In [ ]:
# The inputs are four floats: sepal length, sepal width, petal length, petal width.
inputs  = np.array(iris.data)

# Outputs are 0 (=setosa), 1 (=versicolor) or 2 (=virginica).
outputs = np.array(iris.target)


## Categorical Classification

Here we are using the Keras utility `to_categorical()` to allow us to turn our output categories into binary class matrices. This is often refered to as "One-Hot" encoding. This is for use with categorical_crossentropy and classification of our species (setosa, versicolor and virginica).

Each Species will be represented as a binary class matrix.

- Setosa [1 0 0]
- Versicolor [0 1 0]
- Virginica [0 0 1]

In [ ]:
# Encode the category integers as binary categorical vairables.
outputs_cats = kr.utils.to_categorical(outputs)
outputs_cats

## Divide & Conquer
### Splitting the data

We can now randomly split the data into two sets for:

- Training
- Testing

In [ ]:
# Split the input and output data sets into training and test subsets.
inds = np.random.permutation(len(inputs))
train_inds, test_inds = np.array_split(inds, 2)
inputs_train, outputs_train = inputs[train_inds], outputs_cats[train_inds]
inputs_test,  outputs_test  = inputs[test_inds],  outputs_cats[test_inds]

## Creating a Model

Below we can see an example of a how a Neural Network can be visualized. Every Neural Network is made up of these three main consituents.

- Input Layer
- $x$ number of Hidden Layers
- Output Layer

![neural_net](https://github.com/damiannolan/iris-neural-network/blob/master/img/neural_net.jpeg?raw=1)

### Keras Models

Keras offers a very useful and high level API to handle creation of Neural Networks. The [Keras Sequential Model](https://keras.io/getting-started/sequential-model-guide/) is defined as a *linear stack of layers*. This is perfect for what we need to create an Artificial Neural Network consisting of Input, Output and Hidden nodes. We define our Model and add the layers to it.

We are trying to create a model that will look somewhat similar to below:

![iris_model](https://github.com/damiannolan/iris-neural-network/blob/master/img/iris_model.png?raw=1)

In [ ]:
# Create a neural network.
model = kr.models.Sequential()

# Add an initial layer with 4 input nodes, and a hidden layer with 16 nodes.
model.add(kr.layers.Dense(16, input_shape=(4,)))
# Apply the sigmoid activation function to that layer.
model.add(kr.layers.Activation("sigmoid"))
# Add another layer, connected to the layer with 16 nodes, containing three output nodes.
model.add(kr.layers.Dense(3))
# Use the softmax activation function there.
model.add(kr.layers.Activation("softmax"))

## Activation Functions

An [Activation Function](https://en.wikipedia.org/wiki/Activation_function) in a Neural Network defines the output of a given node given its input or set of inputs. Above we applying two activation functions in separate layers.

### Sigmoid
A sigmoid function is a mathematical function having an "S" shaped curve (sigmoid curve). Often, sigmoid function refers to the special case of the logistic function shown in the first figure and defined by the formula:


![sigmoid](https://github.com/damiannolan/iris-neural-network/blob/master/img/sigmoid.svg?raw=1)

Below we see a plot of the "S" shaped curved or "Sigmoid Curve".

![curve](https://github.com/damiannolan/iris-neural-network/blob/master/img/Logistic-curve.svg.png?raw=1)

It's usage in neural network are:
1. Activation function that transform linear inputs to nonlinear outputs.
2. Bound output to between 0 and 1 so that it can be interpreted as a probability.
3. Make computation easier than arbitrary activation functions.

### Softmax

[Softmax regression](http://ufldl.stanford.edu/tutorial/supervised/SoftmaxRegression/) (or multinomial logistic regression) is a generalization of logistic regression to the case where we want to handle multiple classes.

Softmax regression is defined by the mathematical formula:

![softmax](https://github.com/damiannolan/iris-neural-network/blob/master/img/softmax.svg?raw=1)

Here are using Softmax to allow us to let our data flow through the hidden layers and essentially end up as one of our defined classes:

- Setosa
- Versicolor
- Virginica


In [ ]:
# Display our Model using the summary function
model.summary()

## Configure the Model for training and fit the training data

We configure the Model using the `compile()` function defined in the [Keras Model API](https://keras.io/models/model/).
We define an Optimizer, a Loss function and an additional metric - accuracy.

So before we can use our Model for we must first train it. Using the training data subset which we extracted before we can now fit it to our Model.

The goal here is for the Optimizer to essentially minimize the Loss.

We fit the model passing our inputs and our expected outputs and train it across 100 "Epochs" or training cycles. On each iteration we improve the improve the accuracy and miniize the loss.

In [ ]:
# Configure the model for training.
# Uses the adam optimizer and categorical cross entropy as the loss function.
# Add in some extra metrics - accuracy being the only one.
model.compile(optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"])

# Fit the model using our training data.
model.fit(inputs_train, outputs_train, epochs=100, batch_size=1, verbose=1)

## Evaluate the Loss and Accuracy of the Model

Now that we have trained our Model we can evalate it using the test data which we extracted before. Using `evaluate()` we expect our return values of loss and accuracy for our given Test set.

In [ ]:
# Evaluate the model using the test data set.
loss, accuracy = model.evaluate(inputs_test, outputs_test, verbose=1)

# Output the accuracy of the model.
print("\n\nLoss: %6.4f\tAccuracy: %6.4f" % (loss, accuracy))

## Making predictions using the Model

To make predictions using our Model we must first prepare the input data to be what the model expects. Here we use a couple of Numpy functions such as `around()` and `expand_dims()` to prepare the input data for prediction.

We can then pass get our prediction as a String value from `outputs_vals` which defined earlier in the Notebook.

In [ ]:
# Predict the class of a single flower.
prediction = np.around(model.predict(np.expand_dims(inputs_test[0], axis=0))).astype(int)[0]

print("Actual: %s\tEstimated: %s" % (outputs_test[0].astype(int), prediction))
print("That means it's a %s" % outputs_vals[prediction.astype(bool)][0])

## Saving and Loading the Model

Keras offers a very simplistic way to save and load your model.

In [ ]:
# Save the model to a file for later use.
model.save("iris_neural_network.h5")

We can easily reload the model in another script using `model = load_model("path_to_model.h5")`